In [202]:
# 5-fold Cross Validation
def cross_validation(model, train_input, train_target):
    index = torch.randperm(1000)
    index1, index2, index3, index4, index5 = index[:200], 
    
    
    # 1 fold testing, 4 fold training
    k_fold_train = train_input[], train_input[index[:200]]
    k_fold_target = 
    
    # train the model
    train_model(model, k_fold_train, k_fold_target, mini_batch_size)
    
    # 
    
    return 

In [352]:
index = torch.randperm(1000)

In [200]:
import torch
import math

from torch import optim
from torch import Tensor
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

## Table of Content
1. Baseline: CNN with input as a two-channle image pair
2. Two CNNs with MLP for decision:
3. Siamese CNN with MLP for decision: Two 10 * 1 output are feed in to an MLP
4. Siamese CNN with logical decision: Siamese CNN digit classifier

# Data Generation

In [428]:
def data_preparation():
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)
    train, test = {}, {}
    
    train['input'], test['input'] = train_input, test_input
    train['classes'], test['classes'] = train_classes, test_classes
    train['target'], test['target'] = train_target, test_target
    
    # Normlisation
    mean, std = train['input'].mean(), train['input'].std()
    train['input'] = train['input'].sub_(mean).div_(std)
    test['input'] = test['input'].sub_(mean).div_(std)
    
    return train, test

# 1. CNN Baseline with input as a two-channel image pair

In [361]:
class CNN_baseline(nn.Module):
    def __init__(self, nb_hidden):
        super(CNN_baseline, self).__init__()
        # The input to the CNN is a single two-channel image
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)
        # The number of params:
        # conv1: 32 * (2 * 3 * 3 + 1) = 608
        # conv2: 64 * (32 * 3 * 3 + 1) = 18,496
        # fc1: (256 + 1) * 200 = 51,400
        # fc2: (200 + 1) * 2 = 402

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x, None, None

In [439]:
def train_model_baseline(model, train, mini_batch_size=10, EPOCHS=25, LR=1e-2):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = LR)
    
    train_input, train_target = train['input'], train['target']
    for e in range(EPOCHS):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output, _, _ = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.item()
            optimizer.step()
        print('epoch: ', e, 'loss: ', sum_loss)

def compute_nb_errors(model, test, mini_batch_size=10):
    nb_errors = 0
    test_input, test_target = test['input'], test['target']
    
    for b in range(0, test_input.size(0), mini_batch_size):
        output, _, _= model(test_input.narrow(0, b, mini_batch_size))
        
        if output.dim() == 1:
            predicted_target = output
        else:
            _, predicted_target = torch.max(output, 1)
            
        for k in range(mini_batch_size):
            if test_target[b + k] != predicted_target[k]:
                nb_errors += 1
                
    print("The number of errors: ", nb_errors)
    print("The test error rate is", (nb_errors * 100) / test['target'].shape[0], "%")
    return nb_errors, nb_errors / test['target'].shape[0]

In [447]:
def train_and_test(rounds, Net_name, training_function):
    error_rates = []
    for i in range(rounds):
        # Generate new data for this iteration
        train, test = data_preparation()
        
        # Create, train and test the model
        model = Net_name(200)
        training_function(model, train)
        nb_errors, error_rate = compute_nb_errors(model, test)
        error_rates.append(error_rate)
    
    error_rates = Tensor(error_rates)
    expecation, st_var = torch.mean(error_rates), torch.std(error_rates)
    print('The average error rate is {}%'.format(expecation.item() * 100)), 
    print('The standard deviation of error rate is {}%'.format(std_var.item()*100))
    
    return expecation, st_var

In [448]:
expecation, std_var = train_and_test(4, CNN_baseline, train_model_baseline)

epoch:  0 loss:  68.14438533782959
epoch:  1 loss:  66.45653504133224
epoch:  2 loss:  64.02794337272644
epoch:  3 loss:  60.36530554294586
epoch:  4 loss:  56.11432111263275
epoch:  5 loss:  52.35640615224838
epoch:  6 loss:  49.023874282836914
epoch:  7 loss:  45.72488583624363
epoch:  8 loss:  42.36789207160473
epoch:  9 loss:  38.91469290852547
epoch:  10 loss:  35.53023076802492
epoch:  11 loss:  32.172389306128025
epoch:  12 loss:  28.904823154211044
epoch:  13 loss:  25.686659652739763
epoch:  14 loss:  22.436636123806238
epoch:  15 loss:  19.30094574764371
epoch:  16 loss:  16.140057461336255
epoch:  17 loss:  13.276187689974904
epoch:  18 loss:  10.6829738272354
epoch:  19 loss:  8.46495668310672
epoch:  20 loss:  6.650110521353781
epoch:  21 loss:  5.133100904058665
epoch:  22 loss:  4.087267011869699
epoch:  23 loss:  3.279530012048781
epoch:  24 loss:  2.6830933284945786
The number of errors:  179
The test error rate is 17.9 %
epoch:  0 loss:  68.613942861557
epoch:  1 loss

# 2. Two CNNs without weight sharing

In [423]:
class CNN_wo_sharing(nn.Module):
    def __init__(self, nb_hidden):
        super(CNN_wo_sharing, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)
        
        self.conv1_ = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2_ = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1_ = nn.Linear(256, nb_hidden)
        self.fc2_ = nn.Linear(nb_hidden, 10)
        
        self.fc3 = nn.Linear(20, 200)
        self.fc4 = nn.Linear(200, 2)

    def aux_left(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x
    
    def aux_right(self, x):
        x = F.relu(F.max_pool2d(self.conv1_(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2_(x), kernel_size=2))
        x = F.relu(self.fc1_(x.view(-1, 256)))
        x = self.fc2_(x)
        return x
    
    def forward(self, x):
        imgs_left, imgs_right = x.narrow(1,0,1), x.narrow(1,1,1)
        output_left, output_right = self.aux_left(imgs_left), self.aux_right(imgs_right) 
        
        x = torch.cat([output_left,output_right],dim=1)
        x = F.relu(self.fc3(x))
        output = self.fc4(x)
        return output, output_left, output_right
    
    
def train_model(model, train, aux_loss=True, mini_batch_size=10, EPOCH=25, LR=15e-2):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = LR)
    
    for e in range(EPOCH):
        epoch_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            
            img_pairs, num_pairs, targets = train['input'].narrow(0, b, mini_batch_size), train['classes'].narrow(0, b, mini_batch_size), train['target'].narrow(0, b, mini_batch_size)
            num_left, num_right = num_pairs[:, 0], num_pairs[:, 1]
            
            result, output_left, output_right = model(img_pairs)
            
            # Compute loss
            if aux_loss:
                final_loss = criterion(result, targets)
                left_loss, right_loss = criterion(output_left, num_left), criterion(output_right, num_right)
                loss = 0.4*final_loss + 0.3*left_loss + 0.3*right_loss
                epoch_loss += loss
            else:
                loss = criterion(output, targets)
                epoch_loss += loss
            
            model.zero_grad()
            loss.backward()
            optimizer.step()
            
        print('epoch ', e, 'loss ', epoch_loss.item())

In [446]:
expecation, std_var = train_and_test(10, CNN_wo_sharing, train_model)

epoch  0 loss  138.0262451171875
epoch  1 loss  70.25399017333984
epoch  2 loss  45.81803894042969
epoch  3 loss  34.82025146484375
epoch  4 loss  23.77640151977539
epoch  5 loss  19.766931533813477
epoch  6 loss  14.92772388458252
epoch  7 loss  12.470576286315918
epoch  8 loss  5.538827419281006
epoch  9 loss  18.37511444091797
epoch  10 loss  5.450985908508301
epoch  11 loss  8.433722496032715
epoch  12 loss  1.9602659940719604
epoch  13 loss  4.9300127029418945
epoch  14 loss  4.295381546020508
epoch  15 loss  5.285189628601074
epoch  16 loss  1.8302180767059326
epoch  17 loss  5.293824672698975
epoch  18 loss  4.157041072845459
epoch  19 loss  5.869750022888184
epoch  20 loss  3.6944122314453125
epoch  21 loss  0.8277783393859863
epoch  22 loss  0.21846678853034973
epoch  23 loss  0.1240498498082161
epoch  24 loss  0.09761348366737366
The number of errors:  59
The test error rate is 5.9 %
epoch  0 loss  142.02304077148438
epoch  1 loss  78.93519592285156
epoch  2 loss  49.13439941

epoch  0 loss  139.53057861328125
epoch  1 loss  73.5126953125
epoch  2 loss  47.4793701171875
epoch  3 loss  37.62995147705078
epoch  4 loss  24.534643173217773
epoch  5 loss  18.15431022644043
epoch  6 loss  14.810086250305176
epoch  7 loss  16.668550491333008
epoch  8 loss  7.75211763381958
epoch  9 loss  7.529784679412842
epoch  10 loss  4.1735615730285645
epoch  11 loss  23.063169479370117
epoch  12 loss  8.979063987731934
epoch  13 loss  3.9659931659698486
epoch  14 loss  1.0608060359954834
epoch  15 loss  3.7394559383392334
epoch  16 loss  9.05819034576416
epoch  17 loss  4.67633581161499
epoch  18 loss  0.7862406373023987
epoch  19 loss  0.27063310146331787
epoch  20 loss  0.16386352479457855
epoch  21 loss  0.12503819167613983
epoch  22 loss  0.10234289616346359
epoch  23 loss  0.08670473843812943
epoch  24 loss  0.07513672858476639
The number of errors:  67
The test error rate is 6.7 %
The average error rate is  6.310000270605087
The standard deviation of error rate is  0.877

# 3. Siamese CNN with FC for decision

In [425]:
class CNN_Siamese_fc(nn.Module):
    def __init__(self, nb_hidden):
        super(CNN_Siamese_fc, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)
        
        self.fc3 = nn.Linear(20, 200)
        self.fc4 = nn.Linear(200, 2)
        # The number of params:
        # conv1: 32 * (2 * 3 * 3 + 1) = 608
        # conv2: 64 * (32 * 3 * 3 + 1) = 18,496
        # fc1: (256 + 1) * 200 = 51,400
        # fc2: (200 + 1) * 2 = 402

    def aux_forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x
    
    def forward(self, x):
        imgs_left, imgs_right = x.narrow(1,0,1), x.narrow(1,1,1)
        output_left, output_right = self.aux_forward(imgs_left), self.aux_forward(imgs_right) 
        
        x = torch.cat([output_left,output_right],dim=1)
        
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x, output_left, output_right

In [449]:
expecation, std_var = train_and_test(10, CNN_Siamese_fc, train_model)

epoch  0 loss  105.39517211914062
epoch  1 loss  42.1317138671875
epoch  2 loss  27.106895446777344
epoch  3 loss  18.853729248046875
epoch  4 loss  16.397180557250977
epoch  5 loss  11.589825630187988
epoch  6 loss  9.36000919342041
epoch  7 loss  9.696049690246582
epoch  8 loss  4.54814338684082
epoch  9 loss  1.5717514753341675
epoch  10 loss  0.536359965801239
epoch  11 loss  0.2679656147956848
epoch  12 loss  0.17014732956886292
epoch  13 loss  0.12859871983528137
epoch  14 loss  0.10337764024734497
epoch  15 loss  0.08610942959785461
epoch  16 loss  0.07364144921302795
epoch  17 loss  0.06422095745801926
epoch  18 loss  0.056969620287418365
epoch  19 loss  0.05120523273944855
epoch  20 loss  0.04652088135480881
epoch  21 loss  0.04262201860547066
epoch  22 loss  0.03930004686117172
epoch  23 loss  0.03649114444851875
epoch  24 loss  0.03403061255812645
The number of errors:  54
The test error rate is 5.4 %
epoch  0 loss  115.98857116699219
epoch  1 loss  44.53904342651367
epoch  

The number of errors:  34
The test error rate is 3.4 %
epoch  0 loss  112.91032409667969
epoch  1 loss  45.41590118408203
epoch  2 loss  26.943300247192383
epoch  3 loss  18.76544189453125
epoch  4 loss  14.036186218261719
epoch  5 loss  12.361778259277344
epoch  6 loss  10.604890823364258
epoch  7 loss  6.418591499328613
epoch  8 loss  13.163365364074707
epoch  9 loss  4.319738864898682
epoch  10 loss  5.903717517852783
epoch  11 loss  1.4521586894989014
epoch  12 loss  0.9222355484962463
epoch  13 loss  0.31406059861183167
epoch  14 loss  0.17775577306747437
epoch  15 loss  0.12427663803100586
epoch  16 loss  0.0984862744808197
epoch  17 loss  0.08194220066070557
epoch  18 loss  0.07042239606380463
epoch  19 loss  0.06188299506902695
epoch  20 loss  0.05519108846783638
epoch  21 loss  0.04978068545460701
epoch  22 loss  0.045345377177000046
epoch  23 loss  0.04165516793727875
epoch  24 loss  0.03845105320215225
The number of errors:  52
The test error rate is 5.2 %
The average error 

# 4. Siamese CNN with Logistic Decision

In [452]:
class CNN_Siamese_logical(nn.Module):
    def __init__(self, nb_hidden):
        super(CNN_Siamese_logical, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)
        # The number of params:
        # conv1: 32 * (2 * 3 * 3 + 1) = 608
        # conv2: 64 * (32 * 3 * 3 + 1) = 18,496
        # fc1: (256 + 1) * 200 = 51,400
        # fc2: (200 + 1) * 2 = 402

    def aux_forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x
    
    def forward(self, x):
        imgs_left, imgs_right = x.narrow(1,0,1), x.narrow(1,1,1)
        
        output_left, output_right = self.aux_forward(imgs_left), self.aux_forward(imgs_right) 
        
        _, nums_left = torch.max(output_left, 1)
        _, nums_right = torch.max(output_right, 1)
        
        result = (nums_left <= nums_right).int()
        
        return result, output_left, output_right

In [421]:
def train_CNN_logical(model, train, aux_loss=True, mini_batch_size=10, EPOCH=25, LR=8e-2):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = LR)
    
    for e in range(EPOCH):
        epoch_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            img_pairs, num_pairs, targets = train['input'].narrow(0, b, mini_batch_size), train['classes'].narrow(0, b, mini_batch_size), train['target'].narrow(0, b, mini_batch_size)
            num_left, num_right = num_pairs[:, 0], num_pairs[:, 1]
            
            result, output_left, output_right = model(img_pairs)
            
            left_loss, right_loss = criterion(output_left, num_left), criterion(output_right, num_right)
            loss = 0.5*left_loss + 0.5*right_loss
        
            epoch_loss += loss
            
            model.zero_grad()
            loss.backward()
            optimizer.step()
            
        print('epoch ', e, 'loss ', epoch_loss.item())

In [451]:
expecation, std_var = train_and_test(10, CNN_Siamese_logical, train_CNN_logical)

epoch  0 loss  148.23110961914062
epoch  1 loss  38.17509078979492
epoch  2 loss  22.812667846679688
epoch  3 loss  13.175081253051758
epoch  4 loss  8.719794273376465
epoch  5 loss  5.581304550170898
epoch  6 loss  3.811678886413574
epoch  7 loss  2.2604057788848877
epoch  8 loss  1.6467983722686768
epoch  9 loss  2.2411293983459473
epoch  10 loss  1.1075817346572876
epoch  11 loss  0.5336564779281616
epoch  12 loss  0.36964067816734314
epoch  13 loss  0.31481215357780457
epoch  14 loss  0.25156423449516296
epoch  15 loss  0.19151468575000763
epoch  16 loss  0.16258719563484192
epoch  17 loss  0.14101165533065796
epoch  18 loss  0.1248413622379303
epoch  19 loss  0.11207697540521622
epoch  20 loss  0.1014941856265068
epoch  21 loss  0.09262632578611374
epoch  22 loss  0.08531440794467926
epoch  23 loss  0.07895655184984207
epoch  24 loss  0.07358299195766449
The number of errors:  35
The test error rate is 3.5 %
epoch  0 loss  149.37200927734375
epoch  1 loss  43.14582061767578
epoch 

epoch  24 loss  0.10180128365755081
The number of errors:  27
The test error rate is 2.7 %
epoch  0 loss  138.65127563476562
epoch  1 loss  35.482872009277344
epoch  2 loss  19.041019439697266
epoch  3 loss  12.27718734741211
epoch  4 loss  8.354621887207031
epoch  5 loss  5.744194030761719
epoch  6 loss  3.64113187789917
epoch  7 loss  2.2308144569396973
epoch  8 loss  1.6571710109710693
epoch  9 loss  1.2072829008102417
epoch  10 loss  0.9206352233886719
epoch  11 loss  0.7328132390975952
epoch  12 loss  0.4586949050426483
epoch  13 loss  0.3256432116031647
epoch  14 loss  0.25313326716423035
epoch  15 loss  0.20784927904605865
epoch  16 loss  0.1769982874393463
epoch  17 loss  0.1549447923898697
epoch  18 loss  0.13752509653568268
epoch  19 loss  0.12377262860536575
epoch  20 loss  0.11233527958393097
epoch  21 loss  0.1029590293765068
epoch  22 loss  0.09470555931329727
epoch  23 loss  0.08779425173997879
epoch  24 loss  0.08161202818155289
The number of errors:  30
The test error 

In [409]:
error_bank=[]
for lr in range(1, 11, 1):
    lr = lr * 1e-2
    print(lr)
    model_CNN_Siamese = CNN_Siamese(200)
    train_CNN_Siamese(model_CNN_Siamese, train, LR=lr)
    nb_errors, error_rate = compute_nb_errors(model_CNN_Siamese, test)
    error_bank.append(error_rate)

0.01
0 221.69261169433594
1 187.79782104492188
2 123.73810577392578
3 79.03684997558594
4 57.48709487915039
5 45.08570098876953
6 37.1663818359375
7 31.666770935058594
8 27.5953311920166
9 24.454246520996094
10 21.940336227416992
11 19.830947875976562
12 18.049240112304688
13 16.5338191986084
14 15.20166015625
15 14.007040023803711
16 12.970592498779297
17 12.021894454956055
18 11.188342094421387
19 10.411924362182617
20 9.692893028259277
21 9.037147521972656
22 8.430542945861816
23 7.874445915222168
24 7.362853527069092
The number of errors:  31
The test error rate is 3.1 %
0.02
0 213.09039306640625
1 124.25763702392578
2 62.93315124511719
3 42.75352478027344
4 32.98081970214844
5 27.12726402282715
6 22.905223846435547
7 19.777568817138672
8 17.256633758544922
9 14.923192977905273
10 12.954240798950195
11 11.195476531982422
12 9.763631820678711
13 8.499537467956543
14 7.398507118225098
15 6.4704203605651855
16 5.650990009307861
17 4.982872009277344
18 4.404115200042725
19 3.9180161952

In [410]:
error_bank

[3.1, 2.6, 2.8, 2.5, 2.6, 3.0, 3.0, 2.2, 2.8, 2.5]

In [131]:
def train_model(model, train_input, train_target, mini_batch_size):
    criterion = nn.CrossEntropyLoss()
    eta = 1e-2
    optimizer = optim.SGD(model.parameters(), lr = eta)
    nb_epochs = 25

    for e in range(nb_epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.item()
            optimizer.step()
            
        print(e, sum_loss, loss.item())

def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if target[b + k] != predicted_classes[k]:
                nb_errors += 1
                
    return nb_errors

### Auxilliary Model Training

In [132]:
def build_auxilliary_xy(x, y):
    X = torch.cat((x[:,0:1], x[:,1:2]), 0)
    Y = torch.cat((y[:, 0], y[:, 1]), 0)
    return X, Y

In [133]:
X_train, Y_train = build_auxilliary_xy(train_input, train_classes)
X_test,  Y_test  = build_auxilliary_xy(test_input, test_classes)

In [134]:
model = Aux_Net(200)
mini_batch_size = 10

train_model(model, X_train, Y_train, mini_batch_size)
nb_test_errors = compute_nb_errors(model, X_test, Y_test, mini_batch_size)

print('test error of Net {:0.2f}% {:d}/{:d}'\
      .format((100 * nb_test_errors) / X_train.size(0), nb_test_errors, X_train.size(0)))

0 410.9867162704468 1.5258368253707886
1 209.33949556946754 0.6278577446937561
2 106.85264430940151 0.32842621207237244
3 74.97817378491163 0.18593381345272064
4 58.760453063994646 0.11041209846735
5 48.55215054191649 0.07085372507572174
6 40.922046007588506 0.04732464998960495
7 34.960084749385715 0.03532044589519501
8 30.13240941800177 0.028448861092329025
9 26.0903702955693 0.02303236909210682
10 22.801839141640812 0.019775576889514923
11 19.974598643137142 0.016987180337309837
12 17.56650630501099 0.01435835100710392
13 15.501927251229063 0.012452512048184872
14 13.699654190335423 0.010522188618779182
15 12.146738531650044 0.009417446330189705
16 10.791247360990383 0.0077000274322927
17 9.605741623439826 0.0072168996557593346
18 8.508747721847612 0.006226191762834787
19 7.5688173967646435 0.005250950343906879
20 6.772938450390939 0.004765016958117485
21 6.055247908574529 0.004072825890034437
22 5.470559758512536 0.003939161542803049
23 4.947569651325466 0.0031773436348885298
24 4.5

## Architecture:

"""
Architeture 1:

                  -> Aux_Net -> 10 probs -> num1 -
                 |                                |
(image1, image2)-                                  -> compare -> 0/1
                 |                                |
                  -> Aux_Net -> 10 probs -> num2 -
                  
"""
                  
Architecture 2:        

                  -> Aux_Net -> 10 probs  -
                 |                         |
(image1, image2)-                           -> MLP -> 0/1
                 |                         |
                  -> Aux_Net -> 10 probs  -
                  
                  
""

In [160]:
def aux_testing(num_classifier, test_input, test_classes, test_target, mini_batch_size):
    result = Tensor().int()
    nb_correct = 0
    
    for b in range(0, test_input.size(0), mini_batch_size):
        img_pairs, num_pairs, gnd_truths = test_input.narrow(0, b, mini_batch_size), test_classes.narrow(0, b, mini_batch_size), test_target.narrow(0, b, mini_batch_size)
        
        images_left, images_right = img_pairs[:, 0:1], img_pairs[:, 1:2]
        
        output_left, output_right = num_classifier(images_left), num_classifier(images_right)
        _, nums_left = torch.max(output_left, 1)
        _, nums_right = torch.max(output_right, 1)
        
        partial_result = (nums_left <= nums_right).int()
        
        result = torch.cat((result, partial_result), 0)

        nb_correct += (partial_result == gnd_truths).int().sum() 
        
    return result, nb_correct

In [189]:
result, nb_correct = aux_testing(model, test_input, test_classes, test_target, 10)
nb_errors = test_input.shape[0] - nb_correct
print('test error when using an auxilliary model {:0.2f}% {:d}/{:d}'\
      .format(100*nb_errors/float(test_input.shape[0]), nb_errors, test_input.size(0)))

test error when using an auxilliary model 3.90% 39/1000


In [199]:
torch.randperm(20)

tensor([15,  0, 19, 17,  4, 13,  3, 10,  1,  5, 16, 12,  6,  9, 18,  8, 11,  7,
         2, 14])